In [ ]:
!pip install geopandas
!sudo apt install python3-rtree
import pandas as pd
import geopandas as gpd

In [ ]:
import numpy as np
from shapely.geometry import shape
import matplotlib.pyplot as plt
from geopandas.tools import geocode
%matplotlib inline


In [130]:
from bokeh.io import output_file, show,output_notebook
from bokeh.models import ColumnDataSource,ColorBar,HoverTool
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.palettes import Spectral6
from bokeh.palettes import Spectral11
import folium
import csv

In [ ]:
# Generate the calFireList from csv file
calFireList = gpd.read_file("California_Fire_Incidents.csv")
output_notebook()

In [ ]:
# Generate the dataframe directly 
calFireGdf = gpd.GeoDataFrame(  
    calFireList, geometry=gpd.points_from_xy(calFireList.Longitude, calFireList.Latitude)
    )

Set and change the projection

In [ ]:
# Change the projection of dataset from WGS84 to projected coordinate system of Pseudo-Mercator
calFireGdf.crs = {'init': 'epsg:4326'}  
calFireGdf = calFireGdf.to_crs({'init': 'epsg:3857'})

Get the county bound map

In [ ]:
calCounty = gpd.read_file("CA_Counties_TIGER2016.shp")

Output the plot map of California Fire


In [ ]:
# Set and print the base map
contig = calCounty      
fig, ax = plt.subplots(1, 1,figsize=(50,50))
contig.plot(ax=ax)
calFireGdf.plot(ax=ax,color="red")
ax.set_title("California Fires");
plt.savefig("California Fire location.png")

In [ ]:
#calFireJoin = gpd.sjoin(calFireGdf, calCounty) 

Count the fire incident in every county

In [ ]:
calFireCountyCt = calFireGdf.groupby("Counties", as_index=False).count()[["Counties", "CalFireIncident"]]  

In [ ]:
calCounty = calCounty.assign(Count = 0) #Add a new colunm with int type

In [ ]:
#Insert the count number of fire for counties back to county map
#calFireCountyCt colunm range
for i in range(58):
  #calCounty colunm range         
  for j in range(58):       
    if calFireCountyCt['Counties'][i] == calCounty['NAME'][j]:
      calCounty['Count'][j] = calFireCountyCt['CalFireIncident'][i]

Turn to Bokeh and print by HTML

In [ ]:
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [ ]:
calCounty.head()

Colunm Data Source

In [90]:
lons, lats = gpd_bokeh(calCounty)
source = ColumnDataSource(data=dict(
    	x=lons, 
    	y=lats,
        county = calCounty['NAME'],
        fireCounts = calCounty['Count']))

Set the Color Mapper

In [191]:
color_mapper = linear_cmap(field_name = 'fireCounts', palette = Spectral11 ,
                           low = min(calCounty['Count']) ,
                           high = max(calCounty['Count']))
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [194]:
map = figure(plot_width=1000, plot_height=1000,title="California Fire Map", tools=TOOLS,)
map.patches('x', 'y', source=source, line_color="white", line_width=0.1, color=color_mapper)

map.select_one(HoverTool).tooltips = [
    ('Fire Counts', '@fireCounts'),
    ('County', '@county')
]

color_bar = ColorBar(color_mapper=color_mapper['transform'], width=16, location=(0,0))
map.add_layout(color_bar, 'right')
output_file("California Fire Map.html")

In [ ]:
show(map)

Risks

In [84]:
#calPop = gpd.read_file("Cal_Cnty_RacePop.shp")
#calCounty = calCounty.assign(Pop = 0) #Add a new colunm with int type
#for i in range(58):         #calPop colunm range
#  for j in range(58):       #calCounty colunm range
#    if calPop['NAME10'][i] == calCounty['NAME'][j]:
#      calCounty['Pop'][j] = calPop['Total'][i]

The fire risk should related to the county area

In [186]:
calCounty['Risk'] = (calCounty['Count'] / round(calCounty['geometry'].area / 10**9, 2)) 

In [187]:
source_risk = ColumnDataSource(data=dict(
    	x=lons, 
    	y=lats,
        county = calCounty['NAME'],
        fireRisk = calCounty['Risk']))

In [188]:
color_mapper_risk = linear_cmap(field_name = 'fireRisk', palette = Spectral11 ,
                           low = min(calCounty['Risk']) ,
                           high = max(calCounty['Risk']))
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [ ]:
map_risk = figure(plot_width=1000, plot_height=1000,title="California Fire Risk Map", tools=TOOLS,)
map_risk.patches('x', 'y', source=source_risk, line_color="white", line_width=0.1, color=color_mapper_risk)

map_risk.select_one(HoverTool).tooltips = [
    ('Fire Risk', '@fireRisk'),
    ('County', '@county')
]

color_bar_risk = ColorBar(color_mapper=color_mapper_risk['transform'], width=16, location=(0,0))
map_risk.add_layout(color_bar_risk, 'right')
output_file("California Fire Risk Map.html")
show(map_risk)